# COMP 579 Assignment 1

1. Do Not Change the Random Seed
The random seed has been set to ensure reproducibility. Please do not modify it.

2. Guidance for the First Question
For the initial question, fill in the blanks under the sections marked as TODO. Follow the provided structure and complete the missing parts.

3. Approach for Subsequent Questions
For the later questions, we expect you to attempt the solutions independently. You can refer to the examples provided in earlier questions to understand how to 
plot figures and implement solutions.

4. Ensure that the plots you produce for later questions are similar in style and format to those shown in the previous examples.

In [3]:
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.core.debugger import set_trace
np.random.seed(40)

plt.rcParams["figure.figsize"]=10,5

## Q1 Simulator for Bernoulli Bandit

In [4]:
class GaussianBandit:
  """
    A class representing a Gaussian multi-armed bandit.

    Attributes
    ----------
    num_arms : int
        Number of arms in the bandit.
    mean : list or np.ndarray
        List of mean rewards for each arm.
    variance : float
        Variance of the rewards for all arms.

    Methods
    -------
    sample(arm_index)
        Samples a reward from the specified arm based on a Gaussian distribution.
    """

  # TODO:
  def __init__(self, num_arms, mean, variance):
    self.num_arms = num_arms
    self.mean = mean
    self.variance = variance

  def sample(self, arm_index):
    chosen_mean = self.mean[arm_index]
    return np.random.normal(chosen_mean,np.sqrt(self.variance))
 

In [5]:
# TODO:
delta = 0.2
num_arms = 3
mean = [0.5,0.5-delta,0.5+delta]
variance = 0.01
num_samples = 50

three_arm_gaussian_bandit = GaussianBandit(num_arms,mean,variance)

# Store the rewards for each arm
action_rewards = []
actions = range(num_arms)

for action in actions:
    # Store 50 samples per action
    rewards = []
    for i in range(num_samples): # Will sample 50 times for the same action
       sample_reward = three_arm_gaussian_bandit.sample(action)
       rewards.append(sample_reward)
        
    action_rewards.append(rewards)


### Graphs

In [6]:
for action in actions:
  fig, ax = plt.subplots()

  # TODO:
  true_value = three_arm_gaussian_bandit.mean[action]
  estimated_value = np.mean(action_rewards[action])

  # draw the line of the true value
  line_true_val = ax.axhline(y = true_value, color = 'b', linestyle = ':', label = "true value, q*")
  # draw the line of the estimated value
  line_est_val = ax.axhline(y = estimated_value, color = 'r', linestyle = '--', label = "estimated value, q")
  # plot the reward samples
  plt_samples, = ax.plot(action_rewards[action], 'o', label = "reward samples")

  ax.set_xlabel("sample number")
  ax.set_ylabel("reward value")
  ax.set_title("Sample reward, estimated and true expected reward over 50 samples for action %s" %action, y=-0.2)

  # show the legend with the labels of the line
  ax.legend(handles=[line_true_val, line_est_val, plt_samples])

## Q2 Estimated Q values

In [7]:
def update(reward_samples, alpha):
  """
  Each call to the function yields the current incremental average of the reward with a fixed learning rate, alpha
  E.g. Initial call returns alpha * reward_samples[0], second call returns prev_val + alpha * (reward_samples[1] - prev_val)
  where prev_val is the value return from the previous call, so on and so forth

  Parameters
  ----------
  reward_samples : array of int
      samples of reward values from one arm of a bandit
  alpha : int
      learning rate parameter for the averaging
  """
  incremental_avg = 0 
  for reward in reward_samples:
    incremental_avg = incremental_avg + alpha * (reward - incremental_avg)
    yield incremental_avg

def updateAvg(reward_samples):
  """
  Each call to the function yields the current incremental average of the reward
  E.g. Inital call returns reward_samples[0], second call returns the average of reward_samples[0] and reward_samples[1], so on and so forth

  Parameters
  ----------
  reward_samples : array of int
      samples of reward values from one arm of a bandit
  """
  rewards = []
  for reward in reward_samples:
    rewards.append(reward)
    yield np.mean(rewards) # not the optimal way you can use the derivation of incremental update formula
    
def updateDecaying(reward_samples, alpha_0=0.5, lambda_=0.01, p=0.5):
    """
    Each call to the function yields the updated estimate of the action value using an
    improved decaying learning rate.

    Parameters
    ----------
    reward_samples : array-like of int or float
        Samples of reward values from one arm of a bandit.
    alpha_0 : float, optional
        The initial learning rate (default is 0.5).
    lambda_ : float, optional
        The decay rate constant (default is 0.01).
    p : float, optional
        The power parameter for controlling decay (default is 0.5).
    """
    
    t = 0
    incremental_avg = 0
    for reward in reward_samples:
      alpha_t = alpha_0/((1+lambda_*t)**p)
      incremental_avg = incremental_avg + alpha_t * (reward - incremental_avg)
      t += 1
      yield incremental_avg

### Graphs

In [8]:
for action in actions:
  fig, ax = plt.subplots()

  # TODO:
  incr_avgs = list(updateAvg(action_rewards[action])) # a list showing the estimated value function when you use incremental average function after each reward
  alpha_1_percent = list(update(action_rewards[action],alpha=0.01)) # a list showing estimated value function when you use learning rate alpha to update the estimate after each reward
  alpha_10_percent = list(update(action_rewards[action],alpha=0.1)) # same as the above one
  alpha_decay = list(updateDecaying(action_rewards[action],alpha_0=0.5, lambda_=0.01, p=0.5)) # # a list showing estimated value function when you use high learning rate which gets smaller after each reward
  true_value = mean[action]

  # draw the true value line
  line_true_val = ax.axhline(y = true_value, color = 'b', linestyle = ':', label = "true value")

  # plot incremental values for averaging, alpha = 0.01, alpha = 0.1
  plt_incr_avgs, = ax.plot(incr_avgs, label = "incremental average")
  plt_alpha_1_percent, = ax.plot(alpha_1_percent, label = r"$\alpha = 0.01$")
  plt_alpha_10_percent, = ax.plot(alpha_10_percent, label = r"$\alpha = 0.1$")
  plt_alpha_decay, = ax.plot(alpha_decay, label = r"$\alpha decay$")

  ax.set_xlabel("sample number")
  ax.set_ylabel("reward value")
  ax.set_title("Incremental estimates and true expected reward values over 50 samples for action %s" %action, y=-0.2)

  # show the legend with the labels of the line
  ax.legend(handles=[line_true_val, plt_incr_avgs, plt_alpha_1_percent, plt_alpha_10_percent, plt_alpha_decay])

We see that alpha = 0.01 is very slow to improve its estimated Q value

## Q3 Effect of $α$ on Estimated Q values

In [9]:
# TODO:
num_samples = 100

# arrays of the data generated from 100 runs
incr_avgs_runs = []
alpha_1_percent_runs = []
alpha_10_percent_runs = []
alpha_decay_runs = []


# TODO:
for run in range(100):
  # arrays of data generated from the 3 actions in 1 run
  sample_incr_avgs_by_actions = []
  sample_alpha_1_percent_by_actions = []
  sample_alpha_10_percent_by_actions = []
  sample_alpha_decay_by_actions = []

  for action in actions:
    
    # rewards = [0] USE THIS IF YOU WANNA HAVE r0 for all runs = 0 
    rewards = []
    for i in range(num_samples): # Will sample 100 times for the same action
        rewards.append(three_arm_gaussian_bandit.sample(action))

    sample_incr_avgs_by_actions.append(list(updateAvg(rewards)))
    sample_alpha_1_percent_by_actions.append(list(update(rewards,alpha=0.01)))
    sample_alpha_10_percent_by_actions.append(list(update(rewards,alpha=0.1)))
    sample_alpha_decay_by_actions.append(list(updateDecaying(rewards,alpha_0=0.5, lambda_=0.01, p=0.5)))

  incr_avgs_runs.append(sample_incr_avgs_by_actions)
  alpha_1_percent_runs.append(sample_alpha_1_percent_by_actions)
  alpha_10_percent_runs.append(sample_alpha_10_percent_by_actions)
  alpha_decay_runs.append(sample_alpha_decay_by_actions)

print(incr_avgs_runs)
# convert to np arrays
incr_avgs_runs = np.asarray(incr_avgs_runs)
alpha_1_percent_runs = np.asarray(alpha_1_percent_runs)
alpha_10_percent_runs = np.asarray(alpha_10_percent_runs)
alpha_decay_runs = np.asarray(alpha_decay_runs)

In [10]:
# Debugging
print(incr_avgs_runs[0])
print(len(incr_avgs_runs))
print(len(incr_avgs_runs[0]))
print(len(incr_avgs_runs[0][0]))

In [11]:

def list_of_ith_rewards_over_runs(runs, action):
  ith_rewards_over_jth_run = []

  for i in range(len(runs)):  # 0 to 99 # len(runs) = number of trials , if you initialize the first r = 0, this wont be enough since your len(run[action]) will be 101
    ith_rewards = []
    for run in runs:  # for each run
      ith_rewards.append(run[action][i])
    ith_rewards_over_jth_run.append(ith_rewards)

  return ith_rewards_over_jth_run

# Run function and check the length
list_of_lists = list_of_ith_rewards_over_runs(incr_avgs_runs, 0)
print(len(list_of_lists))  # Should be 100
print(len(list_of_lists[0])) # should be 101 since we set the reward0 = 0

### Graphs

In [12]:
for action in actions:
  fig, ax = plt.subplots()

  # obtain averaged incremental reward values for averaging, alpha = 0.01, alpha = 0.1 and decay alpha over 100 runs
  # TODO:
  ithrewards_incr_avgs_by_actions = list_of_ith_rewards_over_runs(incr_avgs_runs, action)
  ithrewards_alpha_1_percent_by_actions = list_of_ith_rewards_over_runs(alpha_1_percent_runs, action)
  ithrewards_alpha_10_percent_by_actions = list_of_ith_rewards_over_runs(alpha_10_percent_runs, action)
  ithrewards_alpha_decay_by_actions = list_of_ith_rewards_over_runs(alpha_decay_runs, action)
  
  mean_incr_avgs_by_actions = np.array([np.mean(rewards) for rewards in ithrewards_incr_avgs_by_actions])
  mean_alpha_1_percent_by_actions = np.array([np.mean(rewards) for rewards in ithrewards_alpha_1_percent_by_actions])
  mean_alpha_10_percent_by_actions = np.array([np.mean(rewards) for rewards in ithrewards_alpha_10_percent_by_actions])
  mean_alpha_decay_by_actions = np.array([np.mean(rewards) for rewards in ithrewards_alpha_decay_by_actions])

  true_value = mean[action]

  # obtain the standard deviation for averaging, alpha = 0.01, alpha = 0.1 and decay alpha over 100 runs
  std_incr_avgs_by_actions = np.array([np.std(rewards) for rewards in ithrewards_incr_avgs_by_actions])
  std_alpha_1_percent_by_actions = np.array([np.std(rewards) for rewards in ithrewards_alpha_1_percent_by_actions])
  std_alpha_10_percent_by_actions = np.array([np.std(rewards) for rewards in ithrewards_alpha_10_percent_by_actions])
  std_alpha_decay_by_actions =  np.array([np.std(rewards) for rewards in ithrewards_alpha_decay_by_actions])

  # obtain the standard error for averaging, alpha = 0.01, alpha = 0.1 and decay alpha over 100 runs
  std_err_incr_avgs_by_actions = np.array([sd / np.sqrt(num_samples) for sd in std_incr_avgs_by_actions])
  std_err_alpha_1_percent_by_actions = np.array([sd / np.sqrt(num_samples) for sd in std_alpha_1_percent_by_actions])
  std_err_alpha_10_percent_by_actions = np.array([sd / np.sqrt(num_samples) for sd in std_alpha_10_percent_by_actions])
  std_err_alpha_decay_by_actions = np.array([sd / np.sqrt(num_samples) for sd in std_alpha_decay_by_actions])

  # draw the true value line
  line_true_val = ax.axhline(y = true_value, color = 'b', linestyle = ':', label = "true value")

  # draw the averaged incremental reward values for averaging
  plt_incr_avgs, = ax.plot(mean_incr_avgs_by_actions, label = "incremental average")
  # draw the error bar/area for averaging
  incr_avgs_minus_std_err = mean_incr_avgs_by_actions - std_err_incr_avgs_by_actions
  incr_avgs_plus_std_err = mean_incr_avgs_by_actions + std_err_incr_avgs_by_actions
  ax.fill_between(range(0,100), incr_avgs_minus_std_err, incr_avgs_plus_std_err, alpha=0.3)

  # draw the averaged incremental reward values for alpha = 0.01
  plt_alpha_1_percent, = ax.plot(mean_alpha_1_percent_by_actions, label = "alpha = 0.01")
  # draw the error bar/area for alpha = 0.01
  alpha_1_percent_minus_std_err = mean_alpha_1_percent_by_actions - std_err_alpha_1_percent_by_actions
  alpha_1_percent_plus_std_err = mean_alpha_1_percent_by_actions + std_err_alpha_1_percent_by_actions
  ax.fill_between(range(0,100), alpha_1_percent_minus_std_err, alpha_1_percent_plus_std_err, alpha=0.3)

  # draw the averaged incremental reward values for alpha = 0.1
  plt_alpha_10_percent, = ax.plot(mean_alpha_10_percent_by_actions, label = "alpha = 0.1")
  # draw the error bar/area for alpha = 0.1
  alpha_10_percent_minus_std_err = mean_alpha_10_percent_by_actions - std_err_alpha_10_percent_by_actions
  alpha_10_percent_plus_std_err = mean_alpha_10_percent_by_actions + std_err_alpha_10_percent_by_actions
  ax.fill_between(range(0,100), alpha_10_percent_minus_std_err, alpha_10_percent_plus_std_err, alpha=0.3)

  plt_alpha_decay, = ax.plot(mean_alpha_decay_by_actions, label = "alpha decay")
  alpha_decay_minus_std_err = mean_alpha_decay_by_actions - std_err_alpha_decay_by_actions
  alpha_decay_plus_std_err = mean_alpha_decay_by_actions + std_err_alpha_decay_by_actions
  ax.fill_between(range(0,100), alpha_decay_minus_std_err, alpha_decay_plus_std_err, alpha=0.3)

  ax.set_xlabel("sample number")
  ax.set_ylabel("reward value")
  ax.set_title("Incremental estimates and true expected reward values averaged over 100 runs for action %s" %action, y=-0.2)

  ax.legend(handles=[line_true_val, plt_incr_avgs, plt_alpha_1_percent, plt_alpha_10_percent, plt_alpha_decay])

### Answers
We can see that incremental average starts out the best, however alpha decay quickly catches up to it. Alpha 0.1 also learns relatively quickly, however, we can see that alpha = 0.01 is too slow of a learning rate to catch up with others over 100 trials. 

It is beneficial to learn a decaying learning rate when we know our sample distributions are stable.

If I were to experiment with alpha values, I would try anything between 0.5 to 0.1. Although I would expect 0.5 to be too big of a value which would cause unstability over the prediction since it is not decaying

## Q4 Epsilon-greedy

In [13]:
def epsilon_greedy(bandit, epsilon, alpha = None, num_time_step = 1000, epsilon_decay=False, lambda_=0.001, stationary=True, change_time_step=500, bandit_new_means=[]):
  """Epsilon greedy algorithm for bandit action selection

  Parameters
  ----------
  bandit : bandit class
      A bernoulli bandit attributes num_arms, and method sample
  epsilon: float
      A parameter which determines the probability for a random action to be selected
  alpha: (optional) float
      A parameter which determined the learning rate for averaging. If alpha is none, incremental averaging is used.
      Default is none, corresponding to incremental averaging.

  Returns
  -------
  R_over_t
      a list of instantaneous return over the time steps
  total_R_over_t
      a list of cummulative reward over the time steps
  est_is_best_over_t
      a list of values of 0 and 1 where 1 indicates the estimated best action is the true best action and 0 otherwise for each time step
  l_over_t
      a list of instanteneous regret over the time steps
  total_l_over_t
      a list of cummulative regret over the time steps
  """
  # TODO:

  num_arms = bandit.num_arms


  
  Q_arr = np.zeros(num_arms)  # np.zeros(num_arms)  # Estimated Q-values for each arm
  N_arr = np.zeros(num_arms)  # Count of times each arm is chosen
  total_R = 0 # Total reward
  total_l = 0 # total regret
  actions = range(num_arms)
  # print(bandit.mean, bandit.num_arms)

  opt_value =  np.max(bandit.mean) # optimal value, i.e. the mean reward of the best action
  # print("opt value", opt_value)  
  best_action = np.argmax(bandit.mean) # the arm with the highest mean
  # print("best action", best_action) 

  R_over_t = [] # a list of instantaneous reward over the time steps
  total_R_over_t = [] # a list of cummulative reward over the time steps
  ith_reward_for_jth_arm = [[] for _ in range(num_arms)] # after a 1000 samples [[rewards when chosen arm0],.., [rewards when chosen arm2]
  est_is_best_over_t = [] # a list of values of 0 and 1 where 1 indicates the estimated best action is the true best action and 0 otherwise for each time step
  l_over_t = [] # a list of instanteneous regret over the time steps
  total_l_over_t = [] # a list of cummulative regret over the time steps
  
  
  epsilon_t = epsilon 

  for time_step in range(num_time_step):
    if not stationary and time_step == change_time_step:
        bandit.mean = bandit_new_means
        opt_value =  np.max(bandit.mean)
        best_action = np.argmax(bandit.mean)
        
    if epsilon_decay:
        epsilon_t = epsilon/(1+lambda_*time_step) 
     
    # A_star is the estimated best action, i.e. exploit (is it estimated or actual best action)
    A_star = np.argmax(Q_arr)
    A_random = np.random.choice(actions) # Choose a random action, i.e. explore
    # print(Q_arr, A_star, A_random)
    A = A_random if np.random.uniform() < epsilon_t else A_star  # Chosen action? choose A_star with 1-epsilon probabilty
    if 0 in Q_arr: # Will sample randomly until all of the actions are sampled
      A = A_random
    curr_R = bandit.sample(A)
    N_arr[A] += 1 # increase the number of times that the action A was chosen

    R_over_t.append(curr_R)
    ith_reward_for_jth_arm[A].append(curr_R)

    # Commendted out for Question 9
    # Update Q-values using incremental averaging or constant step-size
    if stationary:
      if alpha is None:
        Q_arr[A] += (curr_R - Q_arr[A]) / N_arr[A]  # Incremental averaging
      else:
        Q_arr[A] += alpha * (curr_R - Q_arr[A])  # Constant step-size update
    else: # additional code for question 9
      alpha = 0.1
      Q_arr[A] += alpha * (curr_R - Q_arr[A])

    total_R += curr_R
    total_R_over_t.append(total_R)
  
    est_is_best = 1 if A == best_action else 0 # will give 1 if selected action (A) is the best action
    est_is_best_over_t.append(est_is_best)
  
    l_t = opt_value - Q_arr[A]  # Instantenous Regret = Best mean - Expected Value of the action
    l_over_t.append(l_t)
  
    total_l += l_t
    total_l_over_t.append(total_l)
    # print(np.min(l_over_t))
    # print(Q_arr)
  return R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t

R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = epsilon_greedy(three_arm_gaussian_bandit, 0.01)

### Graphs

In [14]:
#TODO:
epsilons = [0,1/8,1/4,1/2,1]
decaying_epsilon_params = {'epsilon_0':1/2 , 'lambda_':0.1 }  # Decaying epsilon parameters

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(18, 18))

for epsilon in epsilons + ["decay"]:

  # arrays of the data generated from 100 runs
  R_over_t_runs = []
  total_R_over_t_runs = []
  est_is_best_over_t_runs = []
  l_over_t_runs = []
  total_l_over_t_runs = []

  for run in range(100):
    if epsilon == "decay":
      R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = epsilon_greedy(
          three_arm_gaussian_bandit, 
          decaying_epsilon_params['epsilon_0'], 
          epsilon_decay=True, 
          lambda_=decaying_epsilon_params['lambda_']
      )
    else:
      R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = epsilon_greedy(
          three_arm_gaussian_bandit, 
          epsilon
      )
    R_over_t_runs.append(R_over_t)
    total_R_over_t_runs.append(total_R_over_t)
    est_is_best_over_t_runs.append(est_is_best_over_t)
    l_over_t_runs.append(l_over_t)
    total_l_over_t_runs.append(total_l_over_t)

  R_over_t_runs = np.asarray(R_over_t_runs)
  total_R_over_t_runs = np.asarray(total_R_over_t_runs)
  est_is_best_over_t_runs = np.asarray(est_is_best_over_t_runs)
  l_over_t_runs = np.asarray(l_over_t_runs)
  total_l_over_t_runs = np.asarray(total_l_over_t_runs)

  # plot the mean reward over time

  mean_R_over_t_runs = np.mean(R_over_t_runs, axis=0)
  std_err_R_over_t_runs = np.std(R_over_t_runs, axis=0) / np.sqrt(np.size(R_over_t_runs, axis=0))

  axs[0,0].plot(mean_R_over_t_runs, label = r"$\epsilon = %s$" %epsilon)

  R_over_t_minus_std_err = mean_R_over_t_runs - std_err_R_over_t_runs
  R_over_t_plus_std_err = mean_R_over_t_runs  + std_err_R_over_t_runs
  axs[0,0].fill_between(range(0,1000), R_over_t_minus_std_err, R_over_t_plus_std_err, alpha=0.4)
  # axs[0,0].errorbar(range(0,1000), mean_R_over_t_runs, yerr=std_err_R_over_t_runs)

  axs[0,0].legend()
  axs[0,0].set_xlabel("time step")
  axs[0,0].set_ylabel("reward value")
  axs[0,0].set_title("Average Instanteneous Reward Received over Time", y=-0.18)

  # plot the mean cummulative reward over time

  mean_total_R_over_t_runs = np.mean(total_R_over_t_runs, axis=0)
  std_err_total_R_over_t_runs = np.std(total_R_over_t_runs, axis=0) / np.sqrt(np.size(total_R_over_t_runs, axis=0))

  axs[0,1].plot(mean_total_R_over_t_runs, label = r"$\epsilon = %s$" %epsilon)

  total_R_over_t_minus_std_err = mean_total_R_over_t_runs - std_err_total_R_over_t_runs
  total_R_over_t_plus_std_err = mean_total_R_over_t_runs  + std_err_total_R_over_t_runs
  axs[0,1].fill_between(range(0,1000), total_R_over_t_minus_std_err, total_R_over_t_plus_std_err, alpha=0.4)

  axs[0,1].legend()
  axs[0,1].set_xlabel("time step")
  axs[0,1].set_ylabel("reward value")
  axs[0,1].set_title("Average Cummulative Reward Received over Time", y=-0.18)

  #plot the mean percentage of the estimated best action being the first action

  est_is_best_over_t_runs_avgs = np.mean(est_is_best_over_t_runs, axis=0)
  plt_est_is_best_over_t_runs_avgs, = axs[1,0].plot(est_is_best_over_t_runs_avgs, label = r"$\epsilon = %s$" %epsilon)

  axs[1,0].legend()
  axs[1,0].set_xlabel("time step")
  axs[1,0].set_ylabel("percentage")
  axs[1,0].set_title("Percentage of the Estimated Best Action Being the First Action", y=-0.18)

  #plot the mean instantaneous regret over time

  l_over_t_runs_avgs = np.mean(l_over_t_runs, axis=0)
  axs[1,1].plot(l_over_t_runs_avgs, label = r"$\epsilon = %s$" %epsilon)

  axs[1,1].legend()
  axs[1,1].set_xlabel("time step")
  axs[1,1].set_ylabel("regret")
  axs[1,1].set_title("Instantaneous Regret over Time", y=-0.18)

  #plot the total regret over time

  total_l_over_t_runs_avgs = np.mean(total_l_over_t_runs, axis=0)
  axs[2,0].plot(total_l_over_t_runs_avgs, label = r"$\epsilon = %s$" %epsilon)

  axs[2,0].legend()
  axs[2,0].set_xlabel("time step")
  axs[2,0].set_ylabel("regret")
  axs[2,0].set_title("Total Regret up to Time Step t", y=-0.18)

axs[-1, -1].axis('off')

title = r'Graphs  for Epsilon Greedy with Varying Epsilons'
fig.suptitle(title, fontsize=16, y=0.08)

plt.show()

### Answers
Since I intialized the model in a way that it will randomly sample actions unless all actions have non-zero Q value prediction,and the standard deviation for rewards is not big enough to confuse the "model", when we have epsilon 0, it always predicts to most optimal arm, resulting in near-0 regret (Epsilon-decay also learns very fast and on par with epsilon = 0). When I don't initialize the model in this way, as in, there is no condition for every arm to be sampled at least once then epsilon = 0 and epsilon = 1 act the same over 100 of trials. Since epsilon 0 sticks with the first random choice makes.
Besides that we see that as Epsilon increases the total regret increases and total reward decreases. We can attribute this to our arms being stationary with relatively small standard deviations and after a few samples, we could have stopped exploring.

## Q5 Hyperparameters for Epsilon-greedy

To have a plain start, you have been provided with predefined functions for generating plots until now. However, moving forward, you are expected to plot graphs on your own.

### Graphs

In [15]:
import numpy as np
import matplotlib.pyplot as plt

epsilons = [1/8, 1/4]
decaying_epsilon_params = {'epsilon_0': 1/2, 'lambda_': 0.1}
alphas = [0.1, 0.01, 0.001]

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(18, 18))

for epsilon in epsilons + ["decay"]:  # Iterate over epsilon values
  for a in alphas:  # Iterate over alpha values

    # Arrays of data from 100 runs
    R_over_t_runs, total_R_over_t_runs = [], []
    est_is_best_over_t_runs, l_over_t_runs, total_l_over_t_runs = [], [], []

    for run in range(100):
      if epsilon == "decay":
        R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = epsilon_greedy(
          three_arm_gaussian_bandit,
          decaying_epsilon_params['epsilon_0'],
          epsilon_decay=True,
          lambda_=decaying_epsilon_params['lambda_'],
          alpha=a
        )
      else:
        R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = epsilon_greedy(
          three_arm_gaussian_bandit,
          epsilon,
          alpha=a
        )

      # Store results
      R_over_t_runs.append(R_over_t)
      total_R_over_t_runs.append(total_R_over_t)
      est_is_best_over_t_runs.append(est_is_best_over_t)
      l_over_t_runs.append(l_over_t)
      total_l_over_t_runs.append(total_l_over_t)

    # Convert lists to numpy arrays
    R_over_t_runs = np.array(R_over_t_runs)
    total_R_over_t_runs = np.array(total_R_over_t_runs)
    est_is_best_over_t_runs = np.array(est_is_best_over_t_runs)
    l_over_t_runs = np.array(l_over_t_runs)
    total_l_over_t_runs = np.array(total_l_over_t_runs)

    # Check if arrays have valid data
    if R_over_t_runs.size == 0 or total_R_over_t_runs.size == 0:
      print(f"Warning: Empty data for epsilon={epsilon}, alpha={a}")
      continue  # Skip plotting if data is missing

    # Compute means and standard errors
    mean_R_over_t_runs = np.mean(R_over_t_runs, axis=0)
    std_err_R_over_t_runs = np.std(R_over_t_runs, axis=0) / np.sqrt(R_over_t_runs.shape[0])

    mean_total_R_over_t_runs = np.mean(total_R_over_t_runs, axis=0)
    std_err_total_R_over_t_runs = np.std(total_R_over_t_runs, axis=0) / np.sqrt(total_R_over_t_runs.shape[0])

    # Plot: Instantaneous Reward
    axs[0, 0].plot(mean_R_over_t_runs, label=f"$\\epsilon = {epsilon}, \\alpha = {a}$")
    axs[0, 0].fill_between(range(0, 1000), mean_R_over_t_runs - std_err_R_over_t_runs,
                           mean_R_over_t_runs + std_err_R_over_t_runs, alpha=0.3)

    # Plot: Cumulative Reward
    axs[0, 1].plot(mean_total_R_over_t_runs, label=f"$\\epsilon = {epsilon}, \\alpha = {a}$")
    axs[0, 1].fill_between(range(0, 1000), mean_total_R_over_t_runs - std_err_total_R_over_t_runs,
                           mean_total_R_over_t_runs + std_err_total_R_over_t_runs, alpha=0.3)

    # Plot: Estimated Best Action
    axs[1, 0].plot(np.mean(est_is_best_over_t_runs, axis=0), label=f"$\\epsilon = {epsilon}, \\alpha = {a}$")

    # Plot: Instantaneous Regret
    axs[1, 1].plot(np.mean(l_over_t_runs, axis=0), label=f"$\\epsilon = {epsilon}, \\alpha = {a}$")

    # Plot: Total Regret
    axs[2, 0].plot(np.mean(total_l_over_t_runs, axis=0), label=f"$\\epsilon = {epsilon}, \\alpha = {a}$")

# Adjust subplot settings
for ax in axs.flat:
  if len(ax.get_lines()) > 0:  # Only show legend if there are plotted lines
    ax.legend()
  else:
    ax.legend([], [], loc='upper right', handlelength=0)  # Prevents warning

  ax.set_xlabel("Time Step")

axs[0, 0].set_ylabel("Reward Value")
axs[0, 0].set_title("Average Instantaneous Reward over Time")

axs[0, 1].set_ylabel("Reward Value")
axs[0, 1].set_title("Average Cumulative Reward over Time")

axs[1, 0].set_ylabel("Percentage")
axs[1, 0].set_title("Estimated Best Action Selection Percentage")

axs[1, 1].set_ylabel("Regret")
axs[1, 1].set_title("Instantaneous Regret over Time")

axs[2, 0].set_ylabel("Regret")
axs[2, 0].set_title("Total Regret up to Time Step t")

# Hide last unused subplot
axs[2, 1].axis('off')

# Adjust layout
fig.suptitle("Epsilon-Greedy Performance Analysis", fontsize=16, y=0.92)
fig.tight_layout(rect=[0, 0, 1, 0.95])

plt.show()


### Answers
All epsilon values perform better (minimize regret, maximize reward) when they are paired with a higher alpha learning rate, best being alpha = 0.1. Epsilon 0.125 (smallest epsilon) and alpha 0.1 (biggest alpha) gives the best results overall. 

## Q6 Gradient Bandit
[20 points] Write a function that implements the gradient bandit algorithm discussed in class. The
 algorithm should use a baseline to normalize the rewards and update preferences for actions based
 on the gradient ascent formula. Plot the same graphs as above for α = 0.1, α = 0.01, α = 0.001
 and a decaying learning rate defined as αt = α0/
 (1+λt)**p 
, with α0 = 0.5, λ = 0.01, and p = 0.5.
 Explain briefly the behavior you observe.

In [16]:
def gradient_bandit(bandit, alpha = 0.1, alpha_decay = False, num_time_step = 1000, lambda_=0.01, p=0.5, stationary=True, change_time_step=500, bandit_new_means=[]):
  H_t = [0.0 for _ in range(bandit.num_arms)]  # H_t is the preference for each action at time step t, initialized to be 0 for all action at t = 0
  R_avg = 0.0 # Average Reward
  R_total = 0.0 # Total Reward
  Regret_avg = 0.0 
  Regret_total = 0.0
  
  
  alpha_0 = alpha

  opt_value =  np.max(bandit.mean) # optimal value, i.e. the mean reward of the best action
  # print("opt value", opt_value)  
  best_action = np.argmax(bandit.mean) # the arm with the highest mean
  # print("best action", best_action) 
  
  R_over_t = [] # a list of instantaneous reward over the time steps
  total_R_over_t = [] # a list of cummulative reward over the time steps
  est_is_best_over_t = [] # a list of values of 0 and 1 where 1 indicates the estimated best action is the true best action and 0 otherwise for each time step
  l_over_t = [] # a list of instanteneous regret over the time steps
  total_l_over_t = [] # a list of cummulative regret over the time steps

  
  for t in range(1, num_time_step + 1):
    if not stationary and t == change_time_step:
      bandit.mean = bandit_new_means
      opt_value =  np.max(bandit.mean)
      best_action = np.argmax(bandit.mean)
    # look through the preferences
    # get the argmax for the chosen action 
    # if 0.0 in H_t, then chose action randomly

    # Compute action probabilities using standard softmax
    exp_H = [math.exp(h) for h in H_t]
    sum_exp_H = sum(exp_H)
    pi = [e_h / sum_exp_H for e_h in exp_H]

    # Select action A based on probabilities pi
    A = random.choices(range(bandit.num_arms), weights=pi, k=1)[0] # random choices returns a list with the index of the selected probability, e.g. [0] or [2]

    R = bandit.sample(A)
    R_over_t.append(R)
    R_total += R
    total_R_over_t.append(R_total)
    # R_avg += (R - R_avg)/t commented for ninth question
    # 2 lines of code for question 9
    update_factor = (1 / t) if stationary else alpha
    R_avg += update_factor * (R - R_avg)

    Regret = opt_value - R # since we dont know the Q value we use the actual reward to calculate regret
    l_over_t.append(Regret)
    Regret_total += Regret
    total_l_over_t.append(Regret_total)


    est_is_best = 1 if A == best_action else 0 # will give 1 if selected action (A) is the best action
    est_is_best_over_t.append(est_is_best)
    
    # If decaying alpha is used update alpha
    if alpha_decay:
      alpha_t = alpha_0/(1+lambda_*t)**p
    else:
      alpha_t = alpha

    # Update preferences using gradient ascent
    for a in range(bandit.num_arms):
      if a == A:
        H_t[a] += alpha_t * (R - R_avg) * (1 - pi[a])
      else:
        H_t[a] += alpha_t * (R - R_avg) * (0 - pi[a])

  return R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t

### Graphs

In [17]:
#TODO:
alphas = [0.1,0.01,0.001]
# decaying_alpha_params = {'alpha_0': 0.5, 'lambda_': 0.1}  # Decaying alpha parameters

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(18, 18))

for alpha in alphas + ["decay"]:

  # arrays of the data generated from 100 runs
  R_over_t_runs = []
  total_R_over_t_runs = []
  est_is_best_over_t_runs = []
  l_over_t_runs = []
  total_l_over_t_runs = []

  for run in range(100):
    if alpha == "decay":
      R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = gradient_bandit(three_arm_gaussian_bandit, alpha = 0.5, alpha_decay = True, num_time_step = 1000, lambda_=0.1, p=0.5, stationary = True)
    else:
      R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = gradient_bandit(three_arm_gaussian_bandit, alpha = alpha, alpha_decay = False, num_time_step = 1000, lambda_=0.1, p=0.5, stationary = True)
        
    R_over_t_runs.append(R_over_t)
    total_R_over_t_runs.append(total_R_over_t)
    est_is_best_over_t_runs.append(est_is_best_over_t)
    l_over_t_runs.append(l_over_t)
    total_l_over_t_runs.append(total_l_over_t)

  R_over_t_runs = np.asarray(R_over_t_runs)
  total_R_over_t_runs = np.asarray(total_R_over_t_runs)
  est_is_best_over_t_runs = np.asarray(est_is_best_over_t_runs)
  l_over_t_runs = np.asarray(l_over_t_runs)
  total_l_over_t_runs = np.asarray(total_l_over_t_runs)

  # plot the mean reward over time

  mean_R_over_t_runs = np.mean(R_over_t_runs, axis=0)
  # print(mean_R_over_t_runs)
  std_err_R_over_t_runs = np.std(R_over_t_runs, axis=0) / np.sqrt(np.size(R_over_t_runs, axis=0))

  axs[0,0].plot(mean_R_over_t_runs, label = r"$\alpha = %s$" %alpha)

  R_over_t_minus_std_err = mean_R_over_t_runs - std_err_R_over_t_runs
  R_over_t_plus_std_err = mean_R_over_t_runs  + std_err_R_over_t_runs
  axs[0,0].fill_between(range(0,1000), R_over_t_minus_std_err, R_over_t_plus_std_err, alpha=0.4)
  # axs[0,0].errorbar(range(0,1000), mean_R_over_t_runs, yerr=std_err_R_over_t_runs)

  axs[0,0].legend()
  axs[0,0].set_xlabel("time step")
  axs[0,0].set_ylabel("reward value")
  axs[0,0].set_title("Average Instantaneous Reward Received over Time", y=-0.18)

  # plot the mean cummulative reward over time

  mean_total_R_over_t_runs = np.mean(total_R_over_t_runs, axis=0) # different alphas seem to be performing exactly the same
  std_err_total_R_over_t_runs = np.std(total_R_over_t_runs, axis=0) / np.sqrt(np.size(total_R_over_t_runs, axis=0))

  axs[0,1].plot(mean_total_R_over_t_runs, label = r"$\alpha = %s$" %alpha)

  total_R_over_t_minus_std_err = mean_total_R_over_t_runs - std_err_total_R_over_t_runs
  total_R_over_t_plus_std_err = mean_total_R_over_t_runs  + std_err_total_R_over_t_runs
  axs[0,1].fill_between(range(0,1000), total_R_over_t_minus_std_err, total_R_over_t_plus_std_err, alpha=0.4)

  axs[0,1].legend()
  axs[0,1].set_xlabel("time step")
  axs[0,1].set_ylabel("reward value")
  axs[0,1].set_title("Average Cumulative Reward Received over Time", y=-0.18)

  #plot the mean percentage of the estimated best action being the third action

  est_is_best_over_t_runs_avgs = np.mean(est_is_best_over_t_runs, axis=0)
  plt_est_is_best_over_t_runs_avgs, = axs[1,0].plot(est_is_best_over_t_runs_avgs, label = r"$\alpha = %s$" %alpha)

  axs[1,0].legend()
  axs[1,0].set_xlabel("time step")
  axs[1,0].set_ylabel("percentage")
  axs[1,0].set_title("Percentage of Runs where Best Action was Chosen", y=-0.18)

  #plot the mean instantaneous regret over time

  l_over_t_runs_avgs = np.mean(l_over_t_runs, axis=0)
  axs[1,1].plot(l_over_t_runs_avgs, label = r"$\alpha = %s$" %alpha)

  axs[1,1].legend()
  axs[1,1].set_xlabel("time step")
  axs[1,1].set_ylabel("regret")
  axs[1,1].set_title("Instantaneous Regret over Time", y=-0.18)

  #plot the total regret over time

  total_l_over_t_runs_avgs = np.mean(total_l_over_t_runs, axis=0)
  axs[2,0].plot(total_l_over_t_runs_avgs, label = r"$\alpha = %s$" %alpha)

  axs[2,0].legend()
  axs[2,0].set_xlabel("time step")
  axs[2,0].set_ylabel("regret")
  axs[2,0].set_title("Total Regret up to Time Step t", y=-0.18)

axs[-1, -1].axis('off')

title = r'Graphs for Gradient bandit with different alphas and alpha decay'
fig.suptitle(title, fontsize=16, y=0.08)

plt.show()

### Answers
Instantaneous regret gets close to 0 over time faster with higher alpha values or alpha decay. Small alpha values have a hard time "learning" thus the instantaneous regret doesn't really go down over 1000 samples. 

## Q7 Thompson Sampling

In [19]:
def thompson_sampling_gaussian(bandit, num_time_step=1000, stationary=True, change_time_step=500, bandit_new_means=[]):

  num_arms = bandit.num_arms
  variance = bandit.variance

  # Initialize posterior parameters for each arm
  # Assume initial prior mean=0 and precision=1 (variance=1)
  posterior_means = [0.0 for _ in range(num_arms)]
  posterior_precisions = [1.0 for _ in range(num_arms)]  # Precision = 1 / variance

  R_total = 0.0  # Total Reward
  Regret_total = 0.0  # Total Regret

  opt_value = np.max(bandit.mean)  # Optimal mean reward
  best_action = np.argmax(bandit.mean)  # Best arm

  R_over_t = []
  total_R_over_t = []
  est_is_best_over_t = []
  l_over_t = []
  total_l_over_t = []

  for t in range(1, num_time_step + 1):
    if not stationary and t == change_time_step:
      bandit.mean = bandit_new_means
      opt_value =  np.max(bandit.mean)
      best_action = np.argmax(bandit.mean)
    # Sample from the posterior for each arm
    sampled_means = [
      np.random.normal(posterior_means[a], np.sqrt(1 / posterior_precisions[a]))
      for a in range(num_arms)
    ]

    # Select the arm with the highest sampled mean
    A = np.argmax(sampled_means)

    # Pull the selected arm and observe the reward
    R = bandit.sample(A)
    R_over_t.append(R)
    R_total += R
    total_R_over_t.append(R_total)

    # Calculate instantaneous regret using actual reward
    Regret = opt_value - R
    l_over_t.append(Regret)
    Regret_total += Regret
    total_l_over_t.append(Regret_total)

    # Check if the selected action is the best action
    est_is_best = 1 if A == best_action else 0
    est_is_best_over_t.append(est_is_best)

    # Update the posterior for the selected arm
    # Assuming known variance, update posterior mean and precision
    # Using conjugate priors for Gaussian
    # Posterior precision: prior precision + 1/variance
    posterior_precisions[A] = (posterior_precisions[A] + 1 / variance) if stationary else (0.9 * posterior_precisions[A] + 1 / variance) # decay factor that adapts to predictions to unstationary problems, for question 9


    # Posterior mean: (prior precision * prior mean + observed reward / variance) / posterior precision
    posterior_means[A] = (posterior_precisions[A] * posterior_means[A] + R / variance) / posterior_precisions[A]

  return R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t



### Graphs

In [20]:
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(18, 18))

# arrays of the data generated from 100 runs
R_over_t_runs = []
total_R_over_t_runs = []
est_is_best_over_t_runs = []
l_over_t_runs = []
total_l_over_t_runs = []

for run in range(100):
  R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = thompson_sampling_gaussian(three_arm_gaussian_bandit, num_time_step=1000)

  R_over_t_runs.append(R_over_t)
  total_R_over_t_runs.append(total_R_over_t)
  est_is_best_over_t_runs.append(est_is_best_over_t)
  l_over_t_runs.append(l_over_t)
  total_l_over_t_runs.append(total_l_over_t)

R_over_t_runs = np.asarray(R_over_t_runs)
total_R_over_t_runs = np.asarray(total_R_over_t_runs)
est_is_best_over_t_runs = np.asarray(est_is_best_over_t_runs)
l_over_t_runs = np.asarray(l_over_t_runs)
total_l_over_t_runs = np.asarray(total_l_over_t_runs)

# Plot the mean reward over time
mean_R_over_t_runs = np.mean(R_over_t_runs, axis=0)
std_err_R_over_t_runs = np.std(R_over_t_runs, axis=0) / np.sqrt(R_over_t_runs.shape[0])

axs[0,0].plot(mean_R_over_t_runs, label='Mean Reward')
R_over_t_minus_std_err = mean_R_over_t_runs - std_err_R_over_t_runs
R_over_t_plus_std_err = mean_R_over_t_runs + std_err_R_over_t_runs
axs[0,0].fill_between(range(1000), R_over_t_minus_std_err, R_over_t_plus_std_err, alpha=0.4, label='Std Error')
axs[0,0].legend()
axs[0,0].set_xlabel("Time Step")
axs[0,0].set_ylabel("Reward")
axs[0,0].set_title("Average Instantaneous Reward Received over Time", y=-0.18)

# Plot the mean cumulative reward over time
mean_total_R_over_t_runs = np.mean(total_R_over_t_runs, axis=0)
std_err_total_R_over_t_runs = np.std(total_R_over_t_runs, axis=0) / np.sqrt(total_R_over_t_runs.shape[0])

axs[0,1].plot(mean_total_R_over_t_runs, label='Mean Cumulative Reward')
total_R_over_t_minus_std_err = mean_total_R_over_t_runs - std_err_total_R_over_t_runs
total_R_over_t_plus_std_err = mean_total_R_over_t_runs + std_err_total_R_over_t_runs
axs[0,1].fill_between(range(1000), total_R_over_t_minus_std_err, total_R_over_t_plus_std_err, alpha=0.4, label='Std Error')
axs[0,1].legend()
axs[0,1].set_xlabel("Time Step")
axs[0,1].set_ylabel("Cumulative Reward")
axs[0,1].set_title("Average Cumulative Reward Received over Time", y=-0.18)

# Plot the mean percentage of the estimated best action being the best
est_is_best_over_t_runs_avgs = np.mean(est_is_best_over_t_runs, axis=0)
axs[1,0].plot(est_is_best_over_t_runs_avgs, label='Best Action Percentage')

axs[1,0].legend()
axs[1,0].set_xlabel("Time Step")
axs[1,0].set_ylabel("Percentage")
axs[1,0].set_title("Percentage of Runs where Best Action was Chosen", y=-0.18)

# Plot the mean instantaneous regret over time
l_over_t_runs_avgs = np.mean(l_over_t_runs, axis=0)
axs[1,1].plot(l_over_t_runs_avgs, label='Instantaneous Regret')

axs[1,1].legend()
axs[1,1].set_xlabel("Time Step")
axs[1,1].set_ylabel("Regret")
axs[1,1].set_title("Instantaneous Regret over Time", y=-0.18)

# Plot the total regret over time
total_l_over_t_runs_avgs = np.mean(total_l_over_t_runs, axis=0)
axs[2,0].plot(total_l_over_t_runs_avgs, label='Total Regret')

axs[2,0].legend()
axs[2,0].set_xlabel("Time Step")
axs[2,0].set_ylabel("Regret")
axs[2,0].set_title("Total Regret up to Time Step t", y=-0.18)

axs[-1, -1].axis('off')

title = r'Graphs for Thompson Sampling'
fig.suptitle(title, fontsize=16, y=0.08)

plt.show()


### Answers
The algorithm manages to learn which action is the best very quickly and drops its instantenous regret, however, we couldn't get logarithmic total regret which is expected from thomson sampling.

## Q8 Comparison of Algorithms

In [21]:
def choose_algorithms(algorithm, bandit, num_time_step=1000, stationary=True, change_time_step=500, bandit_new_means=[]):

  if algorithm == "epsilon":
    # Epsilon-greedy, Epsilon = 0
    R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t  = epsilon_greedy(bandit, epsilon=0, num_time_step=num_time_step, stationary=True, change_time_step=500, bandit_new_means=bandit_new_means)

  if algorithm == "gradient":
    # Gradient bandit, alpha decay with alpha_0 = 0.5
    R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t  = gradient_bandit(three_arm_gaussian_bandit, alpha = 0.5, alpha_decay = True, num_time_step = 1000, lambda_=0.1, p=0.5,stationary=True, change_time_step=500, bandit_new_means=bandit_new_means)

  if algorithm == "thompson":
    # Thompson sampling
    R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = thompson_sampling_gaussian(bandit, num_time_step=num_time_step, stationary=True, change_time_step=500, bandit_new_means=bandit_new_means)

  return (R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t)

### Graphs

In [22]:
algorithm_list = ["epsilon", "gradient", "thompson"]
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(18, 18))

for algorithm in algorithm_list:

  # arrays of the data generated from 100 runs
  R_over_t_runs = []
  total_R_over_t_runs = []
  est_is_best_over_t_runs = []
  l_over_t_runs = []
  total_l_over_t_runs = []

  for run in range(100):
      R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = choose_algorithms(algorithm, three_arm_gaussian_bandit, num_time_step=1000)

      R_over_t_runs.append(R_over_t)
      total_R_over_t_runs.append(total_R_over_t)
      est_is_best_over_t_runs.append(est_is_best_over_t)
      l_over_t_runs.append(l_over_t)
      total_l_over_t_runs.append(total_l_over_t)

  R_over_t_runs = np.asarray(R_over_t_runs)
  total_R_over_t_runs = np.asarray(total_R_over_t_runs)
  est_is_best_over_t_runs = np.asarray(est_is_best_over_t_runs)
  l_over_t_runs = np.asarray(l_over_t_runs)
  total_l_over_t_runs = np.asarray(total_l_over_t_runs)

  # plot the mean reward over time

  mean_R_over_t_runs = np.mean(R_over_t_runs, axis=0)
  # print(mean_R_over_t_runs)
  std_err_R_over_t_runs = np.std(R_over_t_runs, axis=0) / np.sqrt(np.size(R_over_t_runs, axis=0))

  axs[0,0].plot(mean_R_over_t_runs, label = algorithm)

  R_over_t_minus_std_err = mean_R_over_t_runs - std_err_R_over_t_runs
  R_over_t_plus_std_err = mean_R_over_t_runs  + std_err_R_over_t_runs
  axs[0,0].fill_between(range(0,1000), R_over_t_minus_std_err, R_over_t_plus_std_err, alpha=0.4)
  # axs[0,0].errorbar(range(0,1000), mean_R_over_t_runs, yerr=std_err_R_over_t_runs)

  axs[0,0].legend()
  axs[0,0].set_xlabel("time step")
  axs[0,0].set_ylabel("reward value")
  axs[0,0].set_title("Average Instantaneous Reward Received over Time", y=-0.18)

  # plot the mean cummulative reward over time

  mean_total_R_over_t_runs = np.mean(total_R_over_t_runs, axis=0) # different alphas seem to be performing exactly the same
  std_err_total_R_over_t_runs = np.std(total_R_over_t_runs, axis=0) / np.sqrt(np.size(total_R_over_t_runs, axis=0))

  axs[0,1].plot(mean_total_R_over_t_runs, label = algorithm)

  total_R_over_t_minus_std_err = mean_total_R_over_t_runs - std_err_total_R_over_t_runs
  total_R_over_t_plus_std_err = mean_total_R_over_t_runs  + std_err_total_R_over_t_runs
  axs[0,1].fill_between(range(0,1000), total_R_over_t_minus_std_err, total_R_over_t_plus_std_err, alpha=0.4)

  axs[0,1].legend()
  axs[0,1].set_xlabel("time step")
  axs[0,1].set_ylabel("reward value")
  axs[0,1].set_title("Average Cumulative Reward Received over Time", y=-0.18)

  #plot the mean percentage of the estimated best action being the third action

  est_is_best_over_t_runs_avgs = np.mean(est_is_best_over_t_runs, axis=0)
  plt_est_is_best_over_t_runs_avgs, = axs[1,0].plot(est_is_best_over_t_runs_avgs,label = algorithm)

  axs[1,0].legend()
  axs[1,0].set_xlabel("time step")
  axs[1,0].set_ylabel("percentage")
  axs[1,0].set_title("Percentage of Runs where Best Action was Chosen", y=-0.18)

  #plot the mean instantaneous regret over time

  l_over_t_runs_avgs = np.mean(l_over_t_runs, axis=0)
  axs[1,1].plot(l_over_t_runs_avgs, label = algorithm)

  axs[1,1].legend()
  axs[1,1].set_xlabel("time step")
  axs[1,1].set_ylabel("regret")
  axs[1,1].set_title("Instantaneous Regret over Time", y=-0.18)

  #plot the total regret over time

  total_l_over_t_runs_avgs = np.mean(total_l_over_t_runs, axis=0)
  axs[2,0].plot(total_l_over_t_runs_avgs, label = algorithm)

  axs[2,0].legend()
  axs[2,0].set_xlabel("time step")
  axs[2,0].set_ylabel("regret")
  axs[2,0].set_title("Total Regret up to Time Step t", y=-0.18)

axs[-1, -1].axis('off')

title = r'Graphs for Different Algorithms'
fig.suptitle(title, fontsize=16, y=0.08)

plt.show()

### Answers
We chose the best hyperparameter for each algorithm by comparing the average total reward at the end of 1000 timesteps. The best algorithm is epsilon = 0 since our arms are stationary and we have a small standard deviation. Then gradient bandit with alpha-decay (alpha = 0.5, lambda_=0.1, p=0.5,) performs the second best. Both epsilon and gradient bandit algorithms total logarithmic regret lines, while thompson seems to be linear (which is not expected).
When epsilon-greedy finds the best value it sticky with it, however, Gradient Bandit still keeps exploring even though it knows the best action. This behavior explains the fluctuaions in instantaneous regret and reward.

## Q9 Non-stationary Environment


In [28]:
delta = 0.2
new_mean = [0.5,0.5+2*delta,0.5-2*delta]

algorithm_list = ["epsilon", "gradient", "thompson"]
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(18, 18))

for algorithm in algorithm_list:

  # arrays of the data generated from 100 runs
  R_over_t_runs = []
  total_R_over_t_runs = []
  est_is_best_over_t_runs = []
  l_over_t_runs = []
  total_l_over_t_runs = []

  for run in range(100):
    R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = choose_algorithms(algorithm, three_arm_gaussian_bandit, num_time_step=1000, stationary=False, change_time_step=500, bandit_new_means=new_mean)

    R_over_t_runs.append(R_over_t)
    total_R_over_t_runs.append(total_R_over_t)
    est_is_best_over_t_runs.append(est_is_best_over_t)
    l_over_t_runs.append(l_over_t)
    total_l_over_t_runs.append(total_l_over_t)

  R_over_t_runs = np.asarray(R_over_t_runs)
  total_R_over_t_runs = np.asarray(total_R_over_t_runs)
  # est_is_best_over_t_runs = np.asarray(est_is_best_over_t_runs)
  # l_over_t_runs = np.asarray(l_over_t_runs)
  # total_l_over_t_runs = np.asarray(total_l_over_t_runs)

  # plot the mean reward over time

  mean_R_over_t_runs = np.mean(R_over_t_runs, axis=0)
  # print(mean_R_over_t_runs)
  std_err_R_over_t_runs = np.std(R_over_t_runs, axis=0) / np.sqrt(np.size(R_over_t_runs, axis=0))

  axs[0,0].plot(mean_R_over_t_runs, label = algorithm)

  R_over_t_minus_std_err = mean_R_over_t_runs - std_err_R_over_t_runs
  R_over_t_plus_std_err = mean_R_over_t_runs  + std_err_R_over_t_runs
  axs[0,0].fill_between(range(0,1000), R_over_t_minus_std_err, R_over_t_plus_std_err, alpha=0.4)
  # axs[0,0].errorbar(range(0,1000), mean_R_over_t_runs, yerr=std_err_R_over_t_runs)

  axs[0,0].legend()
  axs[0,0].set_xlabel("time step")
  axs[0,0].set_ylabel("reward value")
  axs[0,0].set_title("Average Instantaneous Reward Received over Time", y=-0.18)

  # plot the mean cummulative reward over time

  mean_total_R_over_t_runs = np.mean(total_R_over_t_runs, axis=0) # different alphas seem to be performing exactly the same
  std_err_total_R_over_t_runs = np.std(total_R_over_t_runs, axis=0) / np.sqrt(np.size(total_R_over_t_runs, axis=0))

  axs[0,1].plot(mean_total_R_over_t_runs, label = algorithm)

  total_R_over_t_minus_std_err = mean_total_R_over_t_runs - std_err_total_R_over_t_runs
  total_R_over_t_plus_std_err = mean_total_R_over_t_runs  + std_err_total_R_over_t_runs
  axs[0,1].fill_between(range(0,1000), total_R_over_t_minus_std_err, total_R_over_t_plus_std_err, alpha=0.4)

  axs[0,1].legend()
  axs[0,1].set_xlabel("time step")
  axs[0,1].set_ylabel("reward value")
  axs[0,1].set_title("Average Cumulative Reward Received over Time", y=-0.18)

axs[-1, -1].axis('off')

title = r'Graphs for Gradient bandit with different alphas and alpha decay'
fig.suptitle(title, fontsize=16, y=0.08)

plt.show()

delta = 0.2
new_mean = [0.5,0.5+2*delta,0.5-2*delta]

algorithm_list = ["epsilon", "gradient", "thompson"]
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(18, 18))

for algorithm in algorithm_list:

  # arrays of the data generated from 100 runs
  R_over_t_runs = []
  total_R_over_t_runs = []
  est_is_best_over_t_runs = []
  l_over_t_runs = []
  total_l_over_t_runs = []

  for run in range(100):
    R_over_t, total_R_over_t, est_is_best_over_t, l_over_t, total_l_over_t = choose_algorithms(algorithm, three_arm_gaussian_bandit, num_time_step=1000, stationary=True, change_time_step=500, bandit_new_means=new_mean)

    R_over_t_runs.append(R_over_t)
    total_R_over_t_runs.append(total_R_over_t)
    est_is_best_over_t_runs.append(est_is_best_over_t)
    l_over_t_runs.append(l_over_t)
    total_l_over_t_runs.append(total_l_over_t)

  R_over_t_runs = np.asarray(R_over_t_runs)
  total_R_over_t_runs = np.asarray(total_R_over_t_runs)
  # est_is_best_over_t_runs = np.asarray(est_is_best_over_t_runs)
  # l_over_t_runs = np.asarray(l_over_t_runs)
  # total_l_over_t_runs = np.asarray(total_l_over_t_runs)

  # plot the mean reward over time

  mean_R_over_t_runs = np.mean(R_over_t_runs, axis=0)
  # print(mean_R_over_t_runs)
  std_err_R_over_t_runs = np.std(R_over_t_runs, axis=0) / np.sqrt(np.size(R_over_t_runs, axis=0))

  axs[0,0].plot(mean_R_over_t_runs, label = algorithm)

  R_over_t_minus_std_err = mean_R_over_t_runs - std_err_R_over_t_runs
  R_over_t_plus_std_err = mean_R_over_t_runs  + std_err_R_over_t_runs
  axs[0,0].fill_between(range(0,1000), R_over_t_minus_std_err, R_over_t_plus_std_err, alpha=0.4)
  # axs[0,0].errorbar(range(0,1000), mean_R_over_t_runs, yerr=std_err_R_over_t_runs)

  axs[0,0].legend()
  axs[0,0].set_xlabel("time step")
  axs[0,0].set_ylabel("reward value")
  axs[0,0].set_title("Average Instantaneous Reward Received over Time", y=-0.18)

  # plot the mean cummulative reward over time

  mean_total_R_over_t_runs = np.mean(total_R_over_t_runs, axis=0) # different alphas seem to be performing exactly the same
  std_err_total_R_over_t_runs = np.std(total_R_over_t_runs, axis=0) / np.sqrt(np.size(total_R_over_t_runs, axis=0))

  axs[0,1].plot(mean_total_R_over_t_runs, label = algorithm)

  total_R_over_t_minus_std_err = mean_total_R_over_t_runs - std_err_total_R_over_t_runs
  total_R_over_t_plus_std_err = mean_total_R_over_t_runs  + std_err_total_R_over_t_runs
  axs[0,1].fill_between(range(0,1000), total_R_over_t_minus_std_err, total_R_over_t_plus_std_err, alpha=0.4)

  axs[0,1].legend()
  axs[0,1].set_xlabel("time step")
  axs[0,1].set_ylabel("reward value")
  axs[0,1].set_title("Average Cumulative Reward Received over Time", y=-0.18)

  # #plot the mean percentage of the estimated best action being the third action
  # 
  # est_is_best_over_t_runs_avgs = np.mean(est_is_best_over_t_runs, axis=0)
  # plt_est_is_best_over_t_runs_avgs, = axs[1,0].plot(est_is_best_over_t_runs_avgs,label = algorithm)
  # 
  # axs[1,0].legend()
  # axs[1,0].set_xlabel("time step")
  # axs[1,0].set_ylabel("percentage")
  # axs[1,0].set_title("Percentage of Runs where Best Action was Chosen", y=-0.18)
  # 
  # #plot the mean instantaneous regret over time
  # 
  # l_over_t_runs_avgs = np.mean(l_over_t_runs, axis=0)
  # axs[1,1].plot(l_over_t_runs_avgs, label = algorithm)
  # 
  # axs[1,1].legend()
  # axs[1,1].set_xlabel("time step")
  # axs[1,1].set_ylabel("regret")
  # axs[1,1].set_title("Instantaneous Regret over Time", y=-0.18)
  # 
  # #plot the total regret over time
  # 
  # total_l_over_t_runs_avgs = np.mean(total_l_over_t_runs, axis=0)
  # axs[2,0].plot(total_l_over_t_runs_avgs, label = algorithm)
  # 
  # axs[2,0].legend()
  # axs[2,0].set_xlabel("time step")
  # axs[2,0].set_ylabel("regret")
  # axs[2,0].set_title("Total Regret up to Time Step t", y=-0.18)

axs[-1, -1].axis('off')

title = r'Graphs for Different Algorithms with Non-stationary Environment'
fig.suptitle(title, fontsize=16, y=0.08)

plt.show()

### Answers